### Scrap TripAdvisor data by selenium
1.Please running on your own jupyter notebook.

2.Please install selenium,BeautifulSoup4,pandas before you run the code:
```
!pip install selenium
!pip install bs4
!pip install pandas
```
3.Please download the webdriver corresponding to your chrome browser version into the same diretory with this notebook.

https://chromedriver.chromium.org/

4.Every time you run the code, data will be crawled according to the input: url and pages.

Here is a document for our total dataset.

https://www.showdoc.com.cn/2091238148777763

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
result_backup=[]
def pages(url,page_number):
    driver = webdriver.Chrome('chromedriver.exe')
    print(f'Logging in...')
    driver.maximize_window()
    driver.get(url)
    time.sleep(3)
    result=[]
    for j in range(page_number):
        try:
            print(j)
            try:
                button=driver.find_element("xpath","/html/body/div[2]/div[2]/div[2]/div[9]/div/div[1]/div[1]/div/div/div[3]/div[3]/div[2]/div[3]/div[1]/div[2]/div/span[1]")
            except:
                try:
                    button=driver.find_element("xpath","/html/body/div[2]/div[2]/div[2]/div[9]/div/div[1]/div[1]/div/div/div[3]/div[3]/div[3]/div[3]/div[1]/div[2]/div/span[1]")
                except:
                    button=driver.find_element("xpath","/html/body/div[2]/div[2]/div[2]/div[9]/div/div[1]/div[1]/div/div/div[3]/div[13]/div/a[2]")
                    button.click()
                    time.sleep(3)
            if button.text=='Read more':
                button.click()

            for i in range(3,13):
                dic={}
                path = '/html/body/div[2]/div[2]/div[2]/div[9]/div/div[1]/div[1]/div/div/div[3]/div[%s]'%str(i)
                comment = driver.find_element("xpath",path)
                soup = BeautifulSoup(comment.get_attribute('innerHTML'), 'html.parser')
                bubble_rating = list(soup.find_all('span',"ui_bubble_rating"))
                aspect=list(soup.find_all('div','hemdC S2 H2 WWOoy'))
                
#                 if int(bubble_rating[0].attrs['class'][1].replace('bubble_',''))>30:
#                     pass
#                 else:
                dic['overall']=bubble_rating[0].attrs['class'][1].replace('bubble_','')
                for i in range(len(aspect)):
                    dic[aspect[i].get_text()] = bubble_rating[i+1].attrs['class'][1].replace('bubble_','')
#                     if i == 0:
#                         dic['overall']=bubble_rating[i].attrs['class'][1].replace('bubble_','')
#                     elif i ==1:
#                         dic['value']=bubble_rating[i].attrs['class'][1].replace('bubble_','')
#                     elif i ==2:
#                         dic['cleanliness']=bubble_rating[i].attrs['class'][1].replace('bubble_','')
#                     elif i ==3:
#                         dic['service']=bubble_rating[i].attrs['class'][1].replace('bubble_','')
                if 'Location' not in dic:
                    pass
                else:
                    soup.find_all('a')[2].get_text()
                    dic['title'] = soup.find_all('a')[2].get_text()
                    dic['review'] = soup.find_all('q')[0].get_text()
                    dic['date'] = soup.find_all('span','teHYY _R Me S4 H3')[0].get_text()
                    result.append(dic)
                    result_backup.append(dic)
            if j ==0:
                button=driver.find_element("xpath","/html/body/div[2]/div[2]/div[2]/div[9]/div/div[1]/div[1]/div/div/div[3]/div[13]/div/a")
                button.click()
            else:
                button=driver.find_element("xpath","/html/body/div[2]/div[2]/div[2]/div[9]/div/div[1]/div[1]/div/div/div[3]/div[13]/div/a[2]")
                button.click()
            time.sleep(3)
        except:
            button=driver.find_element("xpath","/html/body/div[2]/div[2]/div[2]/div[9]/div/div[1]/div[1]/div/div/div[3]/div[13]/div/a[2]")
            button.click()
            time.sleep(3)

    return result





####for running
url = 'https://www.tripadvisor.com/Hotel_Review-g186338-d193166-Reviews-Pembridge_Palace_Hotel-London_England.html'
one_page_data = pages(url,76)
df = pd.DataFrame(data=one_page_data)

df = df[df[['Value','Location','Service','Rooms','Cleanliness','Sleep Quality']].notnull().any(1)]

order = ['overall','title','review','date','Value','Location','Service','Rooms','Cleanliness','Sleep Quality']
df = df[order]

df.to_excel('scrap_data.xlsx',index=False)
